# Load Data from AWS Data Exchange into S3 Bucket

## Obtaining Data

We obtain EOD stock data from AWS Data Exchange and export it to a S3 bucket. Then we format the data for our daily dataset. In this example we use the following data: https://aws.amazon.com/marketplace/pp/prodview-e2aizdzkos266

### Output dataset 

- Contains 20 years of EOD data for one of the top 10 US companies
- The data is saved into the specified S3 bucket as CSV.

```
hist_data_daily/{sym}.csv (columns: sym,open,high,low,close,vol)
```

In [ ]:
# PLEASE SPECIFY YOUR S3BUCKETS
s3bucket_adx=''
s3bucket=''
# symbol
sym='JNJ'

In [ ]:
# copy daily eod data to local
! aws s3 cp s3://{s3bucket_adx}/daily_adjusted_{sym}.csv ./

In [ ]:
import pandas as pd

df = pd.read_csv("daily_adjusted_"+sym+".csv",infer_datetime_format=True, parse_dates=['timestamp'], index_col=['timestamp'])
del df["split_coefficient"]
del df["dividend_amount"]
del df["adjusted_close"]
df.rename(inplace=True,columns={'volume':'vol'})
df.index=df.index.rename('dt')
df['sym']=sym
df = df[['sym', 'open', 'high', 'low', 'close','vol']]
df.sort_index(inplace=True)
df.head()

In [ ]:
df.to_csv(sym+'.csv')

In [ ]:
!aws s3 cp {sym}.csv s3://{s3bucket}/hist_data_daily/
!rm daily_adjusted_{sym}.csv
!rm {sym}.csv